In [14]:
import boto3
import zipfile
import json
import io
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3', 
                  endpoint_url=s3_host, 
                  aws_access_key_id=access_key_id, 
                  aws_secret_access_key=secret_access_key)

buntain = 'buntain'


In [18]:
def process(folder_prefix):
    lang_dict = dict()
    response = s3.list_objects_v2(Bucket=buntain, Prefix=folder_prefix)
    i = 0
    files = []
    for obj in response.get('Contents', []):
        i += 1
        object_key = obj['Key']
        if object_key.endswith('.zip'):
            files.append(object_key)
    print(f"number of files: {i}")
    for file in files:
        zip_object = s3.get_object(Bucket=buntain, Key=file)
        zip_contents = zip_object['Body'].read()
        zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
        for file_info in zip_file.infolist():
            with zip_file.open(file_info) as json_file:
                file_name = file_info.filename
                if not file_name.endswith("-tweet.json"):
                    continue
                try:
                    json_data = json_file.read().decode('utf-8')
                except: 
                    print("this is a text file")
                parsed_data = json.loads(json_data)
                for ind_data in parsed_data:
                    lang = ind_data['tweet']['tweet_language'] 
                    if lang in lang_dict:
                        lang_dict[lang] +=1
                    else:
                        lang_dict[lang] = 1
    return lang_dict

In [19]:
# folder_prefix_lst = ['twitter.tmrc/August_2022/TMRC14_APAC_1/', 'twitter.tmrc/August_2022/TMRC14_APAC_2/', 'twitter.tmrc/October_2022/TMRC15_APAC_3/']
folder_prefix_lst = ['twitter.tmrc/October_2022/TMRC15_APAC_3/']

dict_lst = []
for f in folder_prefix_lst:
    dict_lst.append(process(f))

number of files: 1000


KeyError: 'tweet_language'

In [6]:
sorted_dict_lst = []
for d in dict_lst:
    sorted_dict_lst.append(dict(sorted(d.items(), key=lambda item: item[1], reverse=True)))


In [7]:
for time_df in dict_lst:
    print(time_df)

{'in': 63791, 'fr': 2, 'en': 1351, 'und': 578, 'fi': 17, 'hu': 23, 'es': 57, 'tl': 249, 'ht': 21, 'tr': 37, 'de': 9, 'ja': 16, 'ko': 15, 'cs': 4, 'nl': 16, 'pt': 6, 'it': 7, 'sv': 4, 'cy': 5, 'lt': 5, 'hi': 10, 'da': 9, 'et': 21, 'ro': 5, 'is': 5, 'eu': 4, 'vi': 2, 'no': 6, 'pl': 6, 'th': 1, 'sl': 1, 'ar': 1, 'ta': 1, 'fa': 1, 'zh': 1, 'lv': 1}
{'en': 176524, 'und': 72625, 'es': 409, 'ur': 3901, 'hi': 14077, 'in': 2130, 'fr': 213, 'et': 492, 'tr': 171, 'mr': 104, 'tl': 1202, 'eu': 43, 'cy': 61, 'pt': 84, 'fi': 150, 'da': 226, 'fa': 211, 'ht': 189, 'ta': 110, 'ca': 116, 'de': 177, 'it': 63, 'pl': 73, 'lt': 40, 'cs': 22, 'no': 41, 'is': 9, 'ps': 1, 'lv': 40, 'sd': 16, 'bn': 366, 'ar': 80, 'ro': 100, 'sl': 23, 'sv': 63, 'nl': 125, 'ne': 44, 'hu': 31, 'te': 4, 'ru': 2, 'ja': 2, 'or': 1, 'gu': 10, 'pa': 7, 'zh': 3, 'vi': 8, 'iw': 2, 'ml': 16, 'kn': 1}
{'und': 469826, 'ur': 1516325, 'en': 1754952, 'fa': 40804, 'nl': 11663, 'tl': 84529, 'in': 220929, 'hi': 168894, 'et': 27108, 'ca': 4400, 'ht

In [9]:
df_lst = []
for d in sorted_dict_lst:
    df_lst.append(pd.DataFrame(d.items(), columns=["language", "count"]))

In [13]:
## downloads data
i = 1
for time_df in df_lst:
    time_df.to_csv(f"~/Coding/buntain/language_data/TMRC14_APAC_{i}.csv", index=False)
    i+=1